# Usual Imports and Setups

In [ ]:
from google.colab import drive
import os
import sys

drive.mount("/content/drive")

# install dependencies:
!pip install pyyaml==5.1
!pip install torch==1.8.0 torchvision==0.9.0

import torch
import torchvision

print(torch.__version__, torch.cuda.is_available())
!gcc --version
# install detectron2: (colab has CUDA 10.1 + torch 1.8)
# See https://detectron2.readthedocs.io/tutorials/install.html for instructions
assert torch.__version__.startswith("1.8")
!pip install detectron2==0.4 -q -f https://dl.fbaipublicfiles.com/detectron2/wheels/cu101/torch1.8/index.html
# Colab's "restart runtime" is needed after installation
!pip install -q -U albumentations

os.chdir("drive/MyDrive/PG/pg_iss/")
assert os.getcwd().split("/")[-1] == "pg_iss"
# clone the repo at the same version, in order to access pre-defined configs
# !git clone --branch v0.4 https://github.com/facebookresearch/detectron2.git detectron2_repo
!pip install -q -e detectron2_repo/projects/TensorMask/

In [ ]:
%load_ext autoreload
%autoreload 2

from detectron2.config import get_cfg
from detectron2.utils import comm
from detectron2.evaluation import COCOEvaluator, verify_results
from detectron2.data.datasets import register_coco_instances
from detectron2.data import MetadataCatalog, DatasetCatalog
from detectron2.utils.visualizer import ColorMode, Visualizer
from detectron2.engine import DefaultPredictor, DefaultTrainer
from detectron2.evaluation import COCOEvaluator, inference_on_dataset
from detectron2.data import build_detection_test_loader
import random, cv2, glob
import numpy as np
from google.colab.patches import cv2_imshow

sys.path.append("detectron2_repo/projects/TensorMask")

from lib import coco_handler, copy_and_paste_augm, constants
from lib.detectron2_utils import *
from lib.copy_and_paste_detectron import *

from tensormask import add_tensormask_config
import tensormask

import warnings

warnings.filterwarnings("ignore", "(Possibly )?corrupt EXIF data", UserWarning)
os.environ["LRU_CACHE_CAPACITY"] = "1"

In [ ]:
cfg = get_cfg()
add_tensormask_config(cfg)
add_cap_config(cfg)
cfg.merge_from_file("configs/test_config.yaml")

In [ ]:
register_coco_instances(
    "train_boxes", {}, "data/split/train/train_anno.json", "data/split/train/imgs/",
)
register_coco_instances(
    "validation_boxes",
    {},
    "data/split/validation/validation_anno.json",
    "data/split/validation/imgs/",
)
register_coco_instances(
    "test_boxes", {}, "data/split/test/test_anno.json", "data/split/test/imgs/",
)
# All
# Scale pyramid

## Change config for testing
Increase number of detections per image and lower the confidence threshold for predictions (both were set more stringend for validtion for performance).

In [ ]:
cfg.TEST.DETECTIONS_PER_IMAGE = 300
cfg.MODEL.SCORE_THRESH_TEST = 0.5
cfg.INPUT.MAX_TEST_SIZE = 3000
cfg.INPUT.MIN_TEST_SIZE = 2500
cfg.OUTPUT_DIR = "./output/cap_base/"
cfg.MODEL.WEIGHTS = os.path.join(cfg.OUTPUT_DIR, "model_final.pth")

# Quantative Testing

## Dedicated test set
Uses the dedicated test set containing instances that were neither used for training nor for validation.

In [ ]:
trainer = BaseTrainer(cfg)
trainer.resume_or_load(resume=True)

In [ ]:
evaluator = COCOEvaluator("test_boxes", ("bbox", "segm"), False, output_dir="./output/")
val_loader = build_detection_test_loader(cfg, "test_boxes")
print(inference_on_dataset(trainer.model, val_loader, evaluator))

## Generated boxes
Test on realistic images generated from the instance pool from the test set.

In [ ]:
# TODO

## Scale Pyramid
Test on different scales in order to estimate generalizability and scale bias

In [ ]:
# TODO

# Qualitative Testing
Use (partially) unannotated images in order to estimate generalizability.

## Inselect boxes

In [ ]:
cfg.TEST.DETECTIONS_PER_IMAGE = 600
cfg.MODEL.TENSOR_MASK.SCORE_THRESH_TEST = 0.5
cfg.MODEL.TENSOR_MASK.NUM_CLASSES = 2
cfg.MODEL.WEIGHTS = os.path.join(cfg.OUTPUT_DIR, "model_final.pth")
predictor = DefaultPredictor(cfg)
cfg.TEST.DETECTIONS_PER_IMAGE = 300

In [ ]:
for path in glob.glob(os.path.join(constants.path_to_data_dir, "InselectTest", "*")):
    if "single" in path:
        continue
    im = cv2.imread(path)
    outputs = predictor(im)
    v = Visualizer(
        im[:, :, ::-1],
        metadata=MetadataCatalog.get("test_boxes"),
        scale=0.5,
        instance_mode=ColorMode.IMAGE_BW,  # remove the colors of unsegmented pixels
    )
    v = v.draw_instance_predictions(outputs["instances"].to("cpu"))
    cv2_imshow(v.get_image()[:, :, ::-1])